Day19


In [1]:
import pandas as pd
import pandas as pd    #用于数据处理和分析，可处理表格数据。
import numpy as np     #用于数值计算，提供了高效的数组操作。
import matplotlib.pyplot as plt    #用于绘制各种类型的图表
import seaborn as sns   #基于matplotlib的高级绘图库，能绘制更美观的统计图形。
import warnings
warnings.filterwarnings("ignore")
 
 # 设置中文字体（解决中文显示问题）
plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows系统常用黑体字体
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号
data = pd.read_csv('heart.csv')    #读取数据
print(data.columns)    #'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 
# 'restecg', 'thalach','exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
#提取连续特征值
continuous_features = ['age','trestbps', 'chol', 'thalach','oldpeak']
print(continuous_features)    #打印出连续特征值的列名
#提取离散特征值
discrete_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal','target']
print(discrete_features)    #打印出离散特征值的列名

#使用映射字典进行转换
mapping_dict = {
    'cp':{0:0,1:1,2:2,3:3},
    'restecg':{0:0,1:1,2:2},
    'slope':{0:0,1:1,2:2},
    'thal':{0:0,1:1,2:2,3:3},
    'ca':{0:0,1:1,2:2,3:3,4:4}
}
for feature, mapping_dict in mapping_dict.items():    #遍历映射字典
    data[feature] = data[feature].map(mapping_dict)    #将映射字典中的值替换为原数据中的值

#对离散特征值进行独热编码
data = pd.get_dummies(data, columns=['sex', 'fbs','exang'])    #对离散特征值进行独热编码
print(data.columns)    #打印出数据的列名
data2 = pd.read_csv("heart.csv")    #读取数据
list_final = []    #新建一个空列表，用于存放独热编码后新增的特征名
for i in data.columns:    #遍历数据的列名
    if i not in data2.columns:    #如果列名不在原数据的列名中
        list_final.append(i)    #将列名添加到列表中
for i in list_final:    #遍历列表中的列名
    data[i] = data[i].astype(int)    #将列名转换为int类型
#划分训练集和测试集
from sklearn.model_selection import train_test_split    #导入train_test_split函数
X = data.drop(['target'], axis=1)    #特征，axis=1表示按列删除
y = data['target']    #标签
#按照8:2划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)    #80%训练集，20%测试集
# print(X_train.shape)    #打印出训练集的形状
# print(X_test.shape)    #打印出测试集的形状
from sklearn.ensemble import RandomForestClassifier #随机森林分类器

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 用于评估分类器性能的指标
from sklearn.metrics import classification_report, confusion_matrix #用于生成分类报告和混淆矩阵
import warnings #用于忽略警告信息
warnings.filterwarnings("ignore") # 忽略所有警告信息
# --- 1. 默认参数的随机森林 ---
# 评估基准模型，这里确实不需要验证集
print("--- 1. 默认参数随机森林 (训练集 -> 测试集) ---")
import time # 这里介绍一个新的库，time库，主要用于时间相关的操作，因为调参需要很长时间，记录下会帮助后人知道大概的时长
start_time = time.time() # 记录开始时间
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train) # 在训练集上训练
rf_pred = rf_model.predict(X_test) # 在测试集上预测
end_time = time.time() # 记录结束时间

print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n默认随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred))
print("默认随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred))


Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')
['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']
Index(['age', 'cp', 'trestbps', 'chol', 'restecg', 'thalach', 'oldpeak',
       'slope', 'ca', 'thal', 'target', 'sex_0', 'sex_1', 'fbs_0', 'fbs_1',
       'exang_0', 'exang_1'],
      dtype='object')
--- 1. 默认参数随机森林 (训练集 -> 测试集) ---
训练与预测耗时: 0.2282 秒

默认随机森林 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        29
           1       0.88      0.88      0.88        32

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61

默认随机森林 在测试集上的混淆矩阵：
[[25  4]
 [ 4 28]]


过去电脑性能比较差，特征数目太多计算起来很慢。同时特征中可能存在很多冗余特征干扰解释性、存在噪声特征干扰精度。

所以在面对高维特征的时候常常需要引入特征降维，我们之前课程中的项目的特征也就小几十个，不太需要做降维，对于某些特征较多的数据，如基因数据、微生物数据、传感器数据等，特征较多，所以会考虑特征降维。

特征降维一般有2种策略：
1. 特征筛选：从n个特征中筛选出m个特征，比如方差筛选，剔除方差过小的特征；利用皮尔逊相关系数筛选；lasso筛选（lasso自带的系数可以理解为重要性）、利用树模型自带的重要性、shap重要性等筛选；特征递归方法
2. 特征组合：从n个特征中组合出m个特征，如pca等

今天这节先说一下特征筛选

特征筛选
## 方差筛选

方差筛选是一种简单而有效的特征筛选方法。它的核心逻辑是：特征的方差反映了数据的变化程度，方差很小的特征几乎没有变化，对模型的预测帮助不大。比如，一个特征的值在所有样本中几乎都一样（方差接近0），那么它对区分不同类别或预测结果几乎没有贡献。因此，方差筛选会设定一个方差阈值，剔除方差低于这个阈值的特征，保留那些变化较大的特征，从而减少特征数量，提高模型效率。

这种方法特别适合处理高维数据，能快速去掉不重要的特征，但它不考虑特征与目标变量之间的关系，可能会误删一些低方差但有意义的特征。


In [2]:
print("---方差筛选（Variance Threshold）---")

from sklearn.feature_selection import VarianceThreshold
import time

start_time = time.time()

selector = VarianceThreshold(threshold=0.01)

X_train_var = selector.fit_transform(X_train)

X_test_var = selector.transform(X_test)

selected_features_var = X_train.columns[selector.get_support()].tolist()

print(f"方差筛选后保留的特征数量: {len(selected_features_var)}")
print(f"保留的特征: {selected_features_var}")

rf_model_var = RandomForestClassifier(random_state=42)

rf_model_var.fit(X_train_var, y_train)

rf_pred_var = rf_model_var.predict(X_test_var)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")

print("\n方差筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_var))

print("方差筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_var))

---方差筛选（Variance Threshold）---
方差筛选后保留的特征数量: 16
保留的特征: ['age', 'cp', 'trestbps', 'chol', 'restecg', 'thalach', 'oldpeak', 'slope', 'ca', 'thal', 'sex_0', 'sex_1', 'fbs_0', 'fbs_1', 'exang_0', 'exang_1']
训练与预测耗时: 0.1510 秒

方差筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        29
           1       0.88      0.88      0.88        32

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61

方差筛选后随机森林在测试集上的混淆矩阵：
[[25  4]
 [ 4 28]]


## 皮尔逊相关系数筛选

皮尔逊相关系数筛选是一种基于特征与目标变量之间相关性的特征选择方法。它的核心逻辑是：计算每个特征与目标变量之间的相关系数（范围在-1到1之间，值越大表示正相关越强，值越小表示负相关越强，接近0表示几乎无关），然后根据相关系数的绝对值大小，选择与目标变量相关性较高的特征，剔除相关性较低的特征。这种方法适用于目标变量是连续型的情况（如果是分类问题，可以先对目标变量编码）。通过皮尔逊相关系数筛选，我们可以保留那些对预测目标最有帮助的特征，减少无关或冗余特征的干扰。

皮尔逊相关系数筛选法是一种基于变量相关性的经典特征选择技术，常用于处理目标变量为连续型的场景。若面对分类问题，通常需要先对目标变量进行编码处理，将其转化为数值型数据后再开展分析。

In [3]:
print("--- 皮尔逊相关系数筛选 ---")
from sklearn.feature_selection import SelectKBest, f_classif
import time

start_time = time.time()

# 计算特征与目标变量的相关性，选择前k个特征（这里设为10个，可调整）
# 注意：皮尔逊相关系数通常用于回归问题（连续型目标变量），但如果目标是分类问题，可以用f_classif
k = 10
selector = SelectKBest(score_func=f_classif, k=k)
X_train_corr = selector.fit_transform(X_train, y_train)
X_test_corr = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_corr = X_train.columns[selector.get_support()].tolist()
print(f"皮尔逊相关系数筛选后保留的特征数量: {len(selected_features_corr)}")
print(f"保留的特征: {selected_features_corr}")

# 训练随机森林模型
rf_model_corr = RandomForestClassifier(random_state=42)
rf_model_corr.fit(X_train_corr, y_train)
rf_pred_corr = rf_model_corr.predict(X_test_corr)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n皮尔逊相关系数筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_corr))
print("皮尔逊相关系数筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_corr))

--- 皮尔逊相关系数筛选 ---
皮尔逊相关系数筛选后保留的特征数量: 10
保留的特征: ['cp', 'thalach', 'oldpeak', 'slope', 'ca', 'thal', 'sex_0', 'sex_1', 'exang_0', 'exang_1']
训练与预测耗时: 0.1461 秒

皮尔逊相关系数筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        29
           1       0.81      0.81      0.81        32

    accuracy                           0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.80      0.80      0.80        61

皮尔逊相关系数筛选后随机森林在测试集上的混淆矩阵：
[[23  6]
 [ 6 26]]


## lasso筛选（基于L1正则化）

Lasso回归（Least Absolute Shrinkage and Selection Operator）是一种结合特征选择和模型训练的方法。它的核心逻辑是：在进行线性回归的同时，通过引入L1正则化项（即惩罚项），强制将一些不重要特征的回归系数压缩到0，从而实现特征筛选。换句话说，Lasso会自动“挑选”对预测目标有贡献的特征（系数不为0），而剔除无关或冗余的特征（系数为0）。这种方法特别适合处理高维数据，可以减少特征数量，提高模型的解释性和计算效率。

In [4]:
print("--- Lasso筛选 (L1正则化) ---")
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import time

start_time = time.time()

# 使用Lasso回归进行特征筛选
lasso = Lasso(alpha=0.01, random_state=42)  # alpha值可调整
selector = SelectFromModel(lasso)
selector.fit(X_train, y_train)
X_train_lasso = selector.transform(X_train)
X_test_lasso = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_lasso = X_train.columns[selector.get_support()].tolist()
print(f"Lasso筛选后保留的特征数量: {len(selected_features_lasso)}")
print(f"保留的特征: {selected_features_lasso}")

# 训练随机森林模型
rf_model_lasso = RandomForestClassifier(random_state=42)
rf_model_lasso.fit(X_train_lasso, y_train)
rf_pred_lasso = rf_model_lasso.predict(X_test_lasso)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nLasso筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_lasso))
print("Lasso筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_lasso))


--- Lasso筛选 (L1正则化) ---
Lasso筛选后保留的特征数量: 12
保留的特征: ['age', 'cp', 'trestbps', 'chol', 'restecg', 'thalach', 'oldpeak', 'slope', 'ca', 'thal', 'sex_0', 'exang_0']
训练与预测耗时: 0.1568 秒

Lasso筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        29
           1       0.84      0.84      0.84        32

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61

Lasso筛选后随机森林在测试集上的混淆矩阵：
[[24  5]
 [ 5 27]]


In [5]:
#树模型重要性
print("--- 树模型自带的重要性筛选 ---")
from sklearn.feature_selection import SelectFromModel
import time

start_time = time.time()

# 使用随机森林的特征重要性进行筛选
rf_selector = RandomForestClassifier(random_state=42)
rf_selector.fit(X_train, y_train)
selector = SelectFromModel(rf_selector, threshold="mean")  # 阈值设为平均重要性，可调整
X_train_rf = selector.transform(X_train)
X_test_rf = selector.transform(X_test)

# 获取筛选后的特征名
selected_features_rf = X_train.columns[selector.get_support()].tolist()
print(f"树模型重要性筛选后保留的特征数量: {len(selected_features_rf)}")
print(f"保留的特征: {selected_features_rf}")

# 训练随机森林模型
rf_model_rf = RandomForestClassifier(random_state=42)
rf_model_rf.fit(X_train_rf, y_train)
rf_pred_rf = rf_model_rf.predict(X_test_rf)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n树模型重要性筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_rf))
print("树模型重要性筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_rf))


--- 树模型自带的重要性筛选 ---
树模型重要性筛选后保留的特征数量: 8
保留的特征: ['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'thal']
训练与预测耗时: 0.2832 秒

树模型重要性筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.85      0.79      0.82        29
           1       0.82      0.88      0.85        32

    accuracy                           0.84        61
   macro avg       0.84      0.83      0.83        61
weighted avg       0.84      0.84      0.84        61

树模型重要性筛选后随机森林在测试集上的混淆矩阵：
[[23  6]
 [ 4 28]]


In [6]:
#shap重要性筛选
print("--- SHAP重要性筛选 ---")
import shap
from sklearn.feature_selection import SelectKBest
import time

start_time = time.time()

# 使用随机森林模型计算SHAP值
rf_shap = RandomForestClassifier(random_state=42)
rf_shap.fit(X_train, y_train)
explainer = shap.TreeExplainer(rf_shap)
shap_values = explainer.shap_values(X_train)

# 计算每个特征的平均SHAP值（取绝对值的平均）
mean_shap = np.abs(shap_values[1]).mean(axis=0)  # shap_values[1]对应正类
k = 10  # 选择前10个特征，可调整
top_k_indices = np.argsort(mean_shap)[-k:]
X_train_shap = X_train.iloc[:, top_k_indices]
X_test_shap = X_test.iloc[:, top_k_indices]

# 获取筛选后的特征名
selected_features_shap = X_train.columns[top_k_indices].tolist()
print(f"SHAP重要性筛选后保留的特征数量: {len(selected_features_shap)}")
print(f"保留的特征: {selected_features_shap}")

# 训练随机森林模型
rf_model_shap = RandomForestClassifier(random_state=42)
rf_model_shap.fit(X_train_shap, y_train)
rf_pred_shap = rf_model_shap.predict(X_test_shap)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nSHAP重要性筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_shap))
print("SHAP重要性筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_shap))


--- SHAP重要性筛选 ---
SHAP重要性筛选后保留的特征数量: 2
保留的特征: ['age', 'cp']
训练与预测耗时: 0.3895 秒

SHAP重要性筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.66      0.86      0.75        29
           1       0.83      0.59      0.69        32

    accuracy                           0.72        61
   macro avg       0.74      0.73      0.72        61
weighted avg       0.75      0.72      0.72        61

SHAP重要性筛选后随机森林在测试集上的混淆矩阵：
[[25  4]
 [13 19]]


In [7]:
#递归特征消除RFE

"""
递归特征消除是一种特征选择方法，广泛用于机器学习中，特别是在分类和回归问题中，
用于从一组特征中筛选出对模型性能贡献最大的子集。RFE的核心思想是通过递归地移除最不重要的特征，
逐步缩小特征集，直到达到预设的特征数量或满足其他停止条件。
"""
print("--- 递归特征消除 (RFE) ---")
from sklearn.feature_selection import RFE
import time

start_time = time.time()

# 使用随机森林作为基础模型进行RFE
base_model = RandomForestClassifier(random_state=42)
rfe = RFE(base_model, n_features_to_select=10)  # 选择10个特征，可调整
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

# 获取筛选后的特征名
selected_features_rfe = X_train.columns[rfe.support_].tolist()
print(f"RFE筛选后保留的特征数量: {len(selected_features_rfe)}")
print(f"保留的特征: {selected_features_rfe}")

# 训练随机森林模型
rf_model_rfe = RandomForestClassifier(random_state=42)
rf_model_rfe.fit(X_train_rfe, y_train)
rf_pred_rfe = rf_model_rfe.predict(X_test_rfe)

end_time = time.time()
print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\nRFE筛选后随机森林在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_rfe))
print("RFE筛选后随机森林在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_rfe))


--- 递归特征消除 (RFE) ---
RFE筛选后保留的特征数量: 10
保留的特征: ['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'thal', 'exang_0', 'exang_1']
训练与预测耗时: 1.1288 秒

RFE筛选后随机森林在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.85      0.79      0.82        29
           1       0.82      0.88      0.85        32

    accuracy                           0.84        61
   macro avg       0.84      0.83      0.83        61
weighted avg       0.84      0.84      0.84        61

RFE筛选后随机森林在测试集上的混淆矩阵：
[[23  6]
 [ 4 28]]
